# ch.3　Python을 이용한 데이터 분석

## 3.9 평균값의 차이 검정

### 3.9.1 2집단 데이터에 대한 t검정
2개의 변수 사이에서 평균값에 차이가 있는지 판단을 하기 위해 t검정 사용  
ex. 약을 먹기 전과 먹은 후 체온차이가 있는지 알아보는 경우  
ex. 큰 바늘로 잡은 물고기와 작은 바늘로 잡은 물고기의 몸길이에 차이가 있는지 알아보는 경우 사용

### 3.9.2 대응표본 t검정
| 피험자 | 약 먹기 전의 체온 | 약 먹은 후의 체온 | 차이 |
| :----: | :---------------: | :---------------: | :--: |
|   A    |       36.2        |       36.8        | 0.6  |
|   B    |       36.2        |       36.1        | -0.1 |
|   C    |       35.3        |       36.8        | 1.5  |
|   D    |       36.1        |       37.1        | 1.0  |
|   E    |       36.1        |       36.9        | 0.8  |  

차이 열의 평균값이 0과 다르면 약을 먹기 전과 후의 체온이 다르다는 주장이가능합니다.
대응표본 t 검정에서는 이와 같이 '차잇값이 0과 유의미한 차이가 있는지'를 가지고 1집단 t검정을 실행합니다.

### 3.9.3 분석 준비

In [12]:
import numpy as np
import pandas as pd
import scipy as sp
from scipy import stats

from matplotlib import pyplot as plt
import seaborn as sns
sns.set()

%precision 3

%matplotlib inline

In [13]:
#데이터 불러오기
paired_test_data = pd.read_csv(
    "3-9-1-paired-t-test.csv")
print(paired_test_data)

  person medicine  body_temperature
0      A   before              36.2
1      B   before              36.2
2      C   before              35.3
3      D   before              36.1
4      E   before              36.1
5      A    after              36.8
6      B    after              36.1
7      C    after              36.8
8      D    after              37.1
9      E    after              36.9


- 귀무가설: 약을 먹기 전과 후의 체온이 변하지 않는다.
- 대립가설: 약을 먹기 전과 후의 체온이 다르다.

### 3.9.4 대응표본 t검정(실습)

In [4]:
# 약을 먹기 전과 후의 표본평균
before = paired_test_data.query(
    'medicine == "before"')["body_temperature"]
after = paired_test_data.query(
    'medicine == "after"')["body_temperature"]
# 배열형으로 변환
before = np.array(before)
after = np.array(after)
# 차이 계산
diff = after - before
diff

array([ 0.6, -0.1,  1.5,  1. ,  0.8])

In [5]:
# 차이값의 평균값이 0괴 다른지 1집단 t검정으로 알아본다.
stats.ttest_1samp(diff, 0)

Ttest_1sampResult(statistic=2.901693483620596, pvalue=0.044043109730074276)

In [9]:
# stats.ttest_rel 함수를 이용하면 간단하게 검정가능하다.
stats.ttest_rel(after, before)

Ttest_relResult(statistic=2.901693483620596, pvalue=0.044043109730074276)

p값이 0.05보다 작으므로 약을 먹기 전과 후의 체온은 유의미한 차이가 있다고 할 수 있다. (귀무가설 기각)

### 3.9.5 독립표본 t검정
독립표본 t검정은 평균값의 차이에 주목한다.  
이 '데이터의 차이'를 보고 1집단 t검정을 한다.  
1집단 t검정의 경우
(표본평균-비교대상값)/표준오차  
로 계산하지만 변수 x와 y가 있는 평균값의 차이를 검정할 땐  
식이 조금 더 복잡해진다.
$$
t=\frac{\mu_x-\mu_y}{\sqrt{{\sigma^2_x/m} + {\sigma^2_y/n}}}
$$

x의 표본평균과 y의 표본평균을 뺀 값을 x의 불편분산과 y의 불편분산을 더한 값의 제곱근으로 나눈다고 생각하면 된다.  
2개의 변수로 분산이 차이가 있다는 것을 가정한 계산을 하기에 분모의 표준오차가 복잡해진다.  

### 3.9.6 독립표본 t검정(실습)

In [10]:
# 평균값
mean_bef = np.mean(before)
mean_aft = np.mean(after)

# 분산
sigma_bef = np.var(before, ddof = 1)
sigma_aft = np.var(after, ddof = 1)

# 샘플사이즈
m = len(before)
n = len(after)

# t값
t_value = (mean_aft - mean_bef) / \
    np.sqrt((sigma_bef/m + sigma_aft/n))
t_value

3.1557282344421034

In [11]:
stats.ttest_ind(after, before, equal_var = False)

Ttest_indResult(statistic=3.1557282344421034, pvalue=0.013484775682079892)

p값이 0.05보다 작으므로 유의미한 차이가 있다고 판단할 수 있다.  
하지만 stats.ttest_rel(after, before)을 사용했을 때와 stats.ttest_ind를 사용했을 때의 p값이 다르다.  
당연한 소리지만, 같은 데이터라도 검정방법이 달라지면 p값이 변한다.

### 3.9.7 독립표본 t 검정 (등분산)
정석방식: 등분산성 검정 후 분산이 다르다고 가정한 t검정과 분산이 같다고 가정한 t검정을 구별한다.  
하지만 등분산에 상관 없이 분산이 다르다고 가정한 t검정을 사용해도 상관없다.  
welch 검정을 사용해도 된다.  
stats.ttest_ind 함수에 equal_var = False를 지정해서 넘기면 분산이 다르다고 가정한 t검정을 시행하는 것이다.

### 3.9.8 p해킹

대응표본 t검정, 독립표본 t검정 외에도 평균값의 차이를 검정하는 방법은 다양하다.  
유의미한 차이를 이끌어내기 위해서 각각의 방법을 달리 할 수 있다.  
하지만 검정 방법을 바꾸는 것이 아닌 데이터를 변환하여 검정하거나 데이터의 일부를 제거하는 등의 방식을 통해 유의미한 차이를 얻어내선 안된다.  
p해킹: p값을 자의적으로 변화시키는 것.